In [1]:
import pandas_data_handler ,database_handler,lookups
import prehook
import lookups 
from misc_handler import return_lookup_items_as_dict
import hook

In [6]:
df_src_list,df_src_titles=prehook.execute_prehook(sql_command_directory_path = 'SQL_COMMANDS')


In [4]:
hook.execute_hook(df_src_titles,df_src_list)

c:\Users\Lenovo\Desktop\Data-Analysis-and-Visualization-of-Heart-Diseases\database_handler.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


In [ ]:
import pandas as pd 
from database_handler import create_connection, execute_query, return_data_as_df
from hook import return_etl_last_updated_date
from logging_handler import show_error_message
from lookups import DestinationDatabase, ETLStep, ErrorHandling, InputTypes
from pandas_data_handler import return_create_statement_from_df, return_insert_into_sql_statement_from_df


def create_insert_sql(db_session, source_name,df_source_list,df_titles,etl_step,input_type=None,etl_date = None):
    
    try:
        source_name = source_name.value
        for df_source, df_title in zip(df_titles,df_source_list):
            dst_table = f"stg_{source_name}_{df_title}"
            if etl_step == ETLStep.PRE_HOOK:
                dataframe_source=return_data_as_df(df_source, input_type)
                create_stmt = return_create_statement_from_df(dataframe_source, 'dw_reporting', dst_table)
                execute_query(db_session=db_session, query= create_stmt)

                # index_name = df_source.index.name.replace(" ", "_").replace("-", "_")
                # create_sql_staging_table_index(db_session, 'dw_reporting', dst_table, index_name)
            elif etl_step == ETLStep.HOOK:
                dataframe_source=return_data_as_df(df_title, input_type)
                latest_date=pd.to_datetime(dataframe_source['last_update']).max()
                etl_date = pd.to_datetime(etl_date)
                if latest_date>etl_date:
                    staging_df=dataframe_source
                if len(staging_df):
                    insert_stmt = return_insert_into_sql_statement_from_df(dataframe_source, 'dw_reporting', dst_table)
                    # execute_query(db_session=db_session, query= insert_stmt)
                    return insert_stmt
    except Exception as error:
        suffix = str(error)
        error_prefix = ErrorHandling.CREATE_INSERT_STAGING_TABLES_ERROR
        show_error_message(error_prefix.value, suffix)
        raise Exception("Error creating/insert into staging tables")




db_session = create_connection()
etl_date, does_etl_time_exists = return_etl_last_updated_date(db_session)
z=create_insert_sql(db_session,DestinationDatabase.DATABASE_NAME,df_src_titles,df_src_list,ETLStep.HOOK,InputTypes.CSV,etl_date)
print(z)

In [3]:
import datetime
from database_handler import execute_query, return_data_as_df
from logging_handler import show_error_message
from lookups import DestinationDatabase, ETLStep, ErrorHandling, studies
from misc_handler import enum_to_lists
from pandas_data_handler import return_create_statement_from_df, return_insert_into_sql_statement_from_df
import pandas as pd
import database_handler
from datetime import datetime


def create_insert_sql(db_session, source_name,df_source_list,df_titles,etl_step,input_type=None,etl_date = None):
    
    try:
        source_name = source_name.value
        for df_source, df_title in zip(df_titles,df_source_list):
            dst_table = f"stg_{source_name}_{df_source}"
            if etl_step == ETLStep.PRE_HOOK:
                dataframe_source=return_data_as_df(df_source, input_type)
                create_stmt = return_create_statement_from_df(dataframe_source, 'dw_reporting', dst_table)
                execute_query(db_session=db_session, query= create_stmt)

                # index_name = df_source.index.name.replace(" ", "_").replace("-", "_")
                # create_sql_staging_table_index(db_session, 'dw_reporting', dst_table, index_name)
            elif etl_step == ETLStep.HOOK:
                dataframe_source=return_data_as_df(df_title, input_type)
                # latest_date=pd.to_datetime(dataframe_source['last_update']).max()
                # etl_date_2=datetime.strptime(etl_date, "%Y-%m-%d")
                # if latest_date<etl_date_2
                staging_df=dataframe_source
                if len(staging_df):
                    insert_stmt = return_insert_into_sql_statement_from_df(dataframe_source, 'dw_reporting', dst_table)
                    execute_query(db_session=db_session, query= insert_stmt)
    except Exception as error:
        suffix = str(error)
        error_prefix = ErrorHandling.CREATE_INSERT_STAGING_TABLES_ERROR
        show_error_message(error_prefix.value, suffix)
        raise Exception("Error creating/insert into staging tables")
etl_date = "2001-04-02"

df_title,source=enum_to_lists(studies)
db_session=database_handler.create_connection()
create_insert_sql(db_session,DestinationDatabase.DATABASE_NAME,source, df_title, ETLStep.HOOK, lookups.InputTypes.CSV,etl_date)

In [3]:
import os
from lookups import ETLStep,DestinationDatabase,ErrorHandling
from database_handler import execute_query


def execute_sql_folder(db_session, sql_command_directory_path, etl_step, target_schema):
    sql_files = [sqlfile for sqlfile in os.listdir(sql_command_directory_path) if sqlfile.endswith('.sql')]
    sorted_sql_files = sorted(sql_files, key=lambda x: int(x[1:x.index('-')]))
    counter = 0
    for sql_file in sorted_sql_files:
        counter+=1
        file_title = sql_file.split('-')
        if file_title[1] == etl_step.value:
            with open(os.path.join(sql_command_directory_path,sql_file), 'r') as file:
                sql_query = file.read()
                sql_query = sql_query.replace('target_schema', target_schema.value)
                return_val = execute_query(db_session= db_session, query= sql_query)
                if not return_val == ErrorHandling.NO_ERROR:
                    raise Exception(f"Error executing SQL File on = " +  str(sql_file))
                
db_session=database_handler.create_connection()
execute_sql_folder(db_session,"SQL_COMMANDS",ETLStep.HOOK,DestinationDatabase.SCHEMA_NAME)
